In [1]:
import torch
from transformers import AutoModelForSeq2SeqLM, BitsAndBytesConfig
from IndicTransTokenizer import IndicProcessor, IndicTransTokenizer

BATCH_SIZE = 4
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
quantization = "4-bit"

/home/abhyuday/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def initialize_model_and_tokenizer(ckpt_dir, direction, quantization):
    if quantization == "4-bit":
        qconfig = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_compute_dtype=torch.bfloat16,
        )
    elif quantization == "8-bit":
        qconfig = BitsAndBytesConfig(
            load_in_8bit=True,
            bnb_8bit_use_double_quant=True,
            bnb_8bit_compute_dtype=torch.bfloat16,
        )
    else:
        qconfig = None

    tokenizer = IndicTransTokenizer(direction=direction)
    model = AutoModelForSeq2SeqLM.from_pretrained(
        ckpt_dir,
        trust_remote_code=True,
        low_cpu_mem_usage=True,
        quantization_config=qconfig,
    )

    if qconfig == None:
        model = model.to(DEVICE)
        if DEVICE == "cuda":
            model.half()

    model.eval()

    return tokenizer, model


def batch_translate(input_sentences, src_lang, tgt_lang, model, tokenizer, ip):
    translations = []
    for i in range(0, len(input_sentences), BATCH_SIZE):
        batch = input_sentences[i : i + BATCH_SIZE]

        # Preprocess the batch and extract entity mappings
        batch = ip.preprocess_batch(batch, src_lang=src_lang, tgt_lang=tgt_lang)

        # Tokenize the batch and generate input encodings
        inputs = tokenizer(
            batch,
            src=True,
            truncation=True,
            padding="longest",
            return_tensors="pt",
            return_attention_mask=True,
        ).to(DEVICE)

        # Generate translations using the model
        with torch.no_grad():
            generated_tokens = model.generate(
                **inputs,
                use_cache=True,
                min_length=0,
                max_length=256,
                num_beams=5,
                num_return_sequences=1,
            )

        # Decode the generated tokens into text
        generated_tokens = tokenizer.batch_decode(generated_tokens.detach().cpu().tolist(), src=False)

        # Postprocess the translations, including entity replacement
        translations += ip.postprocess_batch(generated_tokens, lang=tgt_lang)

        del inputs
        torch.cuda.empty_cache()

    return translations

In [3]:
en_indic_ckpt_dir1 = "ai4bharat/indictrans2-en-indic-dist-200M"
en_indic_tokenizer1, en_indic_model1 = initialize_model_and_tokenizer(en_indic_ckpt_dir1, "en-indic","4-bit" )
indic_en_ckpt_dir = "ai4bharat/indictrans2-indic-en-dist-200M"  # ai4bharat/indictrans2-indic-en-dist-200M
indic_en_tokenizer, indic_en_model = initialize_model_and_tokenizer(indic_en_ckpt_dir, "indic-en", "4-bit")

/tmp/ipykernel_31876/2021851695.py:17: DeprecationWarning: This IndicTransTokenizer is deprecated.
The official Tokenizer is available on HF and can be used as follows:
```
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
```
  tokenizer = IndicTransTokenizer(direction=direction)
/tmp/ipykernel_31876/2021851695.py:17: DeprecationWarning: This IndicTransTokenizer is deprecated.
The official Tokenizer is available on HF and can be used as follows:
```
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
```
  tokenizer = IndicTransTokenizer(direction=direction)


In [4]:
import time

In [5]:
st=time.time()


ip = IndicProcessor(inference=True)

en_sents = [
    "When I was young, I used to go to the park every day. ",
     "He has many old books, which he inherited from his ancestors. ",
     "I can't figure out how to solve my problem. ",
     "She is very hardworking and intelligent, which is why she got all the good marks. ",
    "We watched a new movie last week, which was very inspiring.",
    "If you had met me at that time, we would have gone out to eat.",
    "She went to the market with her sister to buy a new sari.",
    "Raj told me that he is going to his grandmother's house next month.",
    "All the kids were having fun at the party and were eating lots of sweets.",
    "My friend has invited me to his birthday party, and I will give him a gift.",
]

src_lang, tgt_lang = "eng_Latn", "hin_Deva"
hi_translations = batch_translate(en_sents, src_lang, tgt_lang, en_indic_model1, en_indic_tokenizer1, ip)

print(f"\n{src_lang} - {tgt_lang}")
for input_sentence, translation in zip(en_sents, hi_translations):
    print(f"{src_lang}: {input_sentence}")
    print(f"{tgt_lang}: {translation}")
et=time.time()
tt=et-st

print(f"Time taken: {tt:.6f} seconds")
# flush the models to free the GPU memory
#del en_indic_tokenizer, en_indic_model


eng_Latn - hin_Deva
eng_Latn: When I was young, I used to go to the park every day. 
hin_Deva: जब मैं छोटा था, मैं हर दिन पार्क जाता था।
eng_Latn: He has many old books, which he inherited from his ancestors. 
hin_Deva: उनके पास कई पुरानी किताबें हैं, जो उन्हें अपने पूर्वजों से विरासत में मिली हैं।
eng_Latn: I can't figure out how to solve my problem. 
hin_Deva: मुझे समझ नहीं आ रहा है कि मेरी समस्या का समाधान कैसे किया जाए।
eng_Latn: She is very hardworking and intelligent, which is why she got all the good marks. 
hin_Deva: वह बहुत मेहनती और बुद्धिमान हैं, यही कारण है कि उन्हें सभी अच्छे अंक मिले।
eng_Latn: We watched a new movie last week, which was very inspiring.
hin_Deva: हमने पिछले हफ्ते एक नई फिल्म देखी, जो बहुत प्रेरणादायक थी।
eng_Latn: If you had met me at that time, we would have gone out to eat.
hin_Deva: अगर आप मुझसे उस समय मिले होते तो हम बाहर खाना खाने जाते।
eng_Latn: She went to the market with her sister to buy a new sari.
hin_Deva: वह अपनी बहन के साथ नई साड़ी खरीदने क

In [6]:

st=time.time()
hi_sents = [
    "जब मैं छोटा था, मैं हर रोज़ पार्क जाता था।",
    "उसके पास बहुत सारी पुरानी किताबें हैं, जिन्हें उसने अपने दादा-परदादा से विरासत में पाया।",
    "मुझे समझ में नहीं आ रहा कि मैं अपनी समस्या का समाधान कैसे ढूंढूं।",
    "वह बहुत मेहनती और समझदार है, इसलिए उसे सभी अच्छे मार्क्स मिले।",
    "हमने पिछले सप्ताह एक नई फिल्म देखी जो कि बहुत प्रेरणादायक थी।",
    "अगर तुम मुझे उस समय पास मिलते, तो हम बाहर खाना खाने चलते।",
    "वह अपनी दीदी के साथ बाजार गयी थी ताकि वह नई साड़ी खरीद सके।",
    "राज ने मुझसे कहा कि वह अगले महीने अपनी नानी के घर जा रहा है।",
    "सभी बच्चे पार्टी में मज़ा कर रहे थे और खूब सारी मिठाइयाँ खा रहे थे।",
    "मेरे मित्र ने मुझे उसके जन्मदिन की पार्टी में बुलाया है, और मैं उसे एक तोहफा दूंगा।",
]
src_lang, tgt_lang = "hin_Deva", "eng_Latn"
en_translations = batch_translate(hi_sents, src_lang, tgt_lang, indic_en_model, indic_en_tokenizer, ip)


print(f"\n{src_lang} - {tgt_lang}")
for input_sentence, translation in zip(hi_sents, en_translations):
    print(f"{src_lang}: {input_sentence}")
    print(f"{tgt_lang}: {translation}")
et=time.time()
tt=et-st
print(f"Time taken: {tt:.6f} seconds")
# flush the models to free the GPU memory
# del indic_en_tokenizer, indic_en_model


hin_Deva - eng_Latn
hin_Deva: जब मैं छोटा था, मैं हर रोज़ पार्क जाता था।
eng_Latn: When I was a kid, I used to go to the park every day.
hin_Deva: उसके पास बहुत सारी पुरानी किताबें हैं, जिन्हें उसने अपने दादा-परदादा से विरासत में पाया।
eng_Latn: She has a lot of old books, which she inherited from her grandparents.
hin_Deva: मुझे समझ में नहीं आ रहा कि मैं अपनी समस्या का समाधान कैसे ढूंढूं।
eng_Latn: I don't know how to find a solution to my problem.
hin_Deva: वह बहुत मेहनती और समझदार है, इसलिए उसे सभी अच्छे मार्क्स मिले।
eng_Latn: He is very hardworking and sensible, so he got all the good marks.
hin_Deva: हमने पिछले सप्ताह एक नई फिल्म देखी जो कि बहुत प्रेरणादायक थी।
eng_Latn: We saw a new film last week which was very inspiring.
hin_Deva: अगर तुम मुझे उस समय पास मिलते, तो हम बाहर खाना खाने चलते।
eng_Latn: If you meet me at that time, we go out to eat.
hin_Deva: वह अपनी दीदी के साथ बाजार गयी थी ताकि वह नई साड़ी खरीद सके।
eng_Latn: She had gone to the market with her sister-in-law so t

In [1]:
"asdffadfadfs.adfsfafdasFDasfd".split(".")

['asdffadfadfs', 'adfsfafdasFDasfd']

In [26]:
def translate(text,in_lang,out_lang):

    lang={"en-IN":"eng_Latn","hi-IN":"hin_Deva","kn-IN":"kan_Knda","bn-IN":"ben_Beng"}
    in_lang=lang[in_lang]
    out_lang=lang[out_lang]
    if in_lang=="eng_Latn":
        text_split=text.split(".")
        out=[]
        for i in text_split:
            if i!=".":
                out.append(i)
        out.remove("")
        return "".join(batch_translate(out, in_lang, out_lang, en_indic_model1, en_indic_tokenizer1, ip))
        
    else:
        if in_lang=="hi-IN":
            text_split=text.split("।")
            out=[]
            for i in text_split:
                if i!="।":
                    out.append(i)
            out=list(filter("",out))
            out.remove("")
            return "".join(batch_translate(out, in_lang, out_lang, indic_en_model, indic_en_tokenizer, ip))

In [27]:
translate("Birds, encompassing over 10,000 species, exhibit remarkable diversity and adaptability, thriving in environments from arid deserts to icy polar regions. Their forms range from the vivid plumage of parrots and peacocks to the streamlined bodies of falcons and hawks. Birds like the Arctic Tern undertake epic migrations spanning thousands of miles, while Emperor Penguins endure the harsh Antarctic winter through cooperative huddling. Avian vocalizations vary from the melodic songs of nightingales to the intricate mimicry of lyrebirds, and their diets range from nectar-feeding hummingbirds to predatory eagles. Birds play crucial ecological roles as pollinators, seed dispersers, and pest controllers. Despite their resilience, many face threats from habitat loss, climate change, and pollution, making conservation vital. Advances in ornithology, utilizing technologies like satellite tracking and bioacoustics, continue to reveal fascinating insights into their behaviors and migrations, underscoring the importance of preserving these incredible creatures for future generations.","en-IN","hi-IN")

'पक्षी, 10,000 से अधिक प्रजातियों को शामिल करते हुए, उल्लेखनीय विविधता और अनुकूलन क्षमता का प्रदर्शन करते हैं, जो शुष्क रेगिस्तान से लेकर बर्फीले ध्रुवीय क्षेत्रों तक के वातावरण में फलते-फूलते हैं।उनके रूप तोते और मोर के जीवंत पंखों से लेकर बाज़ों और बाज़ों के सुव्यवस्थित शरीर तक हैं।आर्कटिक टर्न जैसे पक्षी हजारों मील तक फैले महाकाव्य प्रवास करते हैं, जबकि सम्राट पेंगुइन सहकारी भीड़ के माध्यम से कठोर अंटार्कटिक सर्दियों को सहन करते हैं।एवियन गायन नाइटिंगेल्स के मधुर गीतों से लेकर लाइरेबर्ड की जटिल नकल तक भिन्न होते हैं, और उनके आहार में अमृत-फीडिंग हमिंगबर्ड से लेकर हिंसक चील तक शामिल हैं।पक्षी परागणकों, बीज फैलावकों और कीट नियंत्रकों के रूप में महत्वपूर्ण पारिस्थितिक भूमिका निभाते हैं।अपने लचीलेपन के बावजूद, कई लोगों को निवास स्थान के नुकसान, जलवायु परिवर्तन और प्रदूषण के खतरों का सामना करना पड़ता है, जिससे संरक्षण महत्वपूर्ण हो जाता हैउपग्रह ट्रैकिंग और बायोएकॉस्टिक्स जैसी प्रौद्योगिकियों का उपयोग करते हुए पक्षी विज्ञान में प्रगति, उनके व्यवहार और प्रवास में आकर्षक अंतर्दृष्टि को प्र